In [ ]:
import numpy as np
import pandas as pd
from sklearn.decomposition import TruncatedSVD as skTSVD
from cuML import TruncatedSVD as cumlTSVD
import pygdf
import os

# Helper Functions

In [ ]:
from timeit import default_timer

class Timer(object):
    def __init__(self):
        self._timer = default_timer
    
    def __enter__(self):
        self.start()
        return self

    def __exit__(self, *args):
        self.stop()

    def start(self):
        """Start the timer."""
        self.start = self._timer()

    def stop(self):
        """Stop the timer. Calculate the interval in seconds."""
        self.end = self._timer()
        self.interval = self.end - self.start

In [ ]:
import gzip
def load_data(nrows, ncols, cached = 'data/mortgage.npy.gz'):
    if os.path.exists(cached):
        with gzip.open(cached) as f:
            X = np.load(f)
        return X[np.random.randint(0,X.shape[0]-1,nrows),:ncols]
    else:
        return np.random.rand(nrows,ncols)

In [ ]:
def np2pygdf(df):
    # convert numpy array to pygdf dataframe
    pdf = pygdf.DataFrame()
    for c in range(df.shape[1]):
        pdf[c] = df[:,c]
    return pdf

In [ ]:
def array_equal(a,b,threshold=1e-4,with_sign=True):
    a = to_nparray(a)
    b = to_nparray(b)
    if with_sign == False:
        a,b = np.abs(a),np.abs(b)
    res = np.max(np.abs(a-b))<threshold
    return res

def to_nparray(x):
    if isinstance(x,np.ndarray) or isinstance(x,pd.DataFrame):
        return np.array(x)
    elif isinstance(x,np.float64):
        return np.array([x])
    elif isinstance(x,pygdf.DataFrame) or isinstance(x,pygdf.Series):
        return x.to_pandas().values
    return x

# Run tests

In [ ]:
%%time
nrows = 2**20
ncols = 256

X = load_data(nrows,ncols)
print('data',X.shape)

In [ ]:
n_components = 10
random_state = 42
algorithm='arpack'

In [ ]:
%%time
tsvd_sk = skTSVD(n_components=n_components,algorithm=algorithm, 
            random_state=random_state)
result_sk = tsvd_sk.fit_transform(X)

In [ ]:
%%time
X = np2pygdf(X)

In [ ]:
%%time
tsvd_cuml = cumlTSVD(n_components=n_components,algorithm=algorithm, 
            random_state=random_state)
result_cuml = tsvd_cuml.fit_transform(X)

In [ ]:
for attr in ['singular_values_','components_','explained_variance_',
             'explained_variance_ratio_']:
    passed = array_equal(getattr(tsvd_sk,attr),getattr(tsvd_cuml,attr),
        threshold=0.05,with_sign=True)
    message = 'compare tsvd: cuml vs sklearn %s %s'%(attr,'equal' if passed else 'NOT equal')
    print(message)

In [ ]:
passed = array_equal(result_sk,result_cuml
            threshold=0.05,with_sign = False)
message = 'compare tsvd: cuml vs sklearn transformed results %s'%('equal'if passed else 'NOT equal')
print(message)